# Full TFs Perturbation prediction via RegVelo

In [1]:
import numpy as np
import pandas as pd
import torch
import scanpy as sc
import scvelo as scv
import scvi

from regvelo import REGVELOVI
import cellrank as cr

import shutil

from rgv_tools import DATA_DIR,FIG_DIR
from rgv_tools.benchmarking import (
    set_output,
)
from rgv_tools.perturbation import TFScanning,get_list_name

/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/regvelo_t

In [2]:
scvi.settings.seed = 0

[rank: 0] Global seed set to 0


## Constants

In [3]:
DATASET = "zebrafish"

In [4]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / "processed").mkdir(parents=True, exist_ok=True)
    (DATA_DIR / DATASET / "results").mkdir(parents=True, exist_ok=True)
    
SAVE_FIGURES = True
if SAVE_FIGURES:
    (FIG_DIR / DATASET).mkdir(parents=True, exist_ok=True)

## Data Loading

In [5]:
adata = sc.read_h5ad(DATA_DIR / DATASET / "processed" / "adata_preprocessed.h5ad")

## Perturbation screening

In [6]:
## prepare skeleton
W = adata.uns["skeleton"].copy()
W = torch.tensor(np.array(W)).int()

## prepare TF
TF = adata.var_names[adata.var["TF"]]

In [7]:
### repeat run the model to get aggregate performance
terminal_states = [
        "mNC_head_mesenchymal",
        "mNC_arch2",
        "mNC_hox34",
        "Pigment",
    ]

for nrun in range(3):
    print("training model...")
    REGVELOVI.setup_anndata(adata, spliced_layer="Ms", unspliced_layer="Mu")
    vae = REGVELOVI(adata, W=W.T,regulators = TF, soft_constraint = False)

    torch.cuda.empty_cache()
    vae.train()
    
    print("save model...")

    model_name = "rgv_model_" + str(nrun)
    coef_name = "coef_" + str(nrun)
    pval_name = "pval_" + str(nrun)
    
    model = DATA_DIR / DATASET / "processed" / "perturb_repeat_runs" / model_name
    coef_save = DATA_DIR / DATASET / "results" / coef_name
    pval_save = DATA_DIR / DATASET / "results" / pval_name
    
    vae.save(model)
    
    print("inferring perturbation...")
    
    while True:
        try:
            perturb_screening = TFScanning(model,adata,7,"cell_type",terminal_states,TF,0)
            coef = pd.DataFrame(np.array(perturb_screening['coefficient']))
            coef.index = perturb_screening['TF']
            coef.columns = get_list_name(perturb_screening['coefficient'][0])

            pval = pd.DataFrame(np.array(perturb_screening['pvalue']))
            pval.index = perturb_screening['TF']
            pval.columns = get_list_name(perturb_screening['pvalue'][0])

            rows_with_nan = coef.isna().any(axis=1)
            # Set all values in those rows to NaN
            coef.loc[rows_with_nan, :] = np.nan
            pval.loc[rows_with_nan, :] = np.nan

            coef.to_csv(coef_save)
            pval.to_csv(pval_save)

            break
        except:
            # If an error is raised, increment a and try again, and need to recompute double knock-out reults
            print("perturbation screening has error, retraining model...")
            shutil.rmtree(model)
            REGVELOVI.setup_anndata(adata, spliced_layer="Ms", unspliced_layer="Mu")
            vae = REGVELOVI(adata, W=W.T,regulators = TF, soft_constraint = False)
            vae.train()
            print("save model...")
            vae.save(model)

training model...


/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/weixu.wang/miniconda3/envs/regvelo_test/li ...
  rank_zero_warn(
/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/weixu.wang/miniconda3/envs/regvelo_test/li ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/weixu.wang/miniconda3/envs

Epoch 997/1500:  66%|██████▋   | 996/1500 [08:13<03:48,  2.21it/s, v_num=1]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 1279/1500:  85%|████████▌ | 1279/1500 [10:03<01:44,  2.12it/s, v_num=1]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: -2305.778. Signaling Trainer to stop.
save model...
inferring perturbation...
INFO     File                                                                                                      
         /ictstr01/home/icb/weixu.wang/regulatory_velo/data/zebrafish/processed/perturb_repeat_runs/rgv_model_2/mod
         el.pt already downloaded                                                                                  


...

Done nr2f2
INFO     File                                                                                                      
         /ictstr01/home/icb/weixu.wang/regulatory_velo/data/zebrafish/processed/perturb_repeat_runs/rgv_model_2/mod
         el.pt already downloaded                                                                                  


/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/weixu.wang/miniconda3/envs/regvelo_test/li ...
  rank_zero_warn(
100%|██████████| 697/697 [00:00<00:00, 2260.68cell/s]


100%|██████████| 4/4 [00:00<00:00, 39.78/s]


Done nr2f5
INFO     File                                                                                                      
         /ictstr01/home/icb/weixu.wang/regulatory_velo/data/zebrafish/processed/perturb_repeat_runs/rgv_model_2/mod
         el.pt already downloaded                                                                                  


/home/icb/weixu.wang/miniconda3/envs/regvelo_test/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/weixu.wang/miniconda3/envs/regvelo_test/li ...
  rank_zero_warn(
 81%|████████  | 563/697 [00:00<00:00, 1709.32cell/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

